In [3]:
import numpy as np

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

tf.keras.backend.clear_session()  # For easy reset of notebook state.


In [4]:
import pandas as pd

data=pd.read_excel('new_data_2.xlsx',dtype=str,headers=None,names=['A','B','C','class'])
print(data.columns)
A_data = data[['A', 'class']].copy()
B_data = data[['B', 'class']].copy()
C_data = data[['C', 'class']].copy()
A_train=A_data.iloc[:50000,:]
B_train=B_data.iloc[:50000,:]
C_train=C_data.iloc[:50000,:]
A_test=A_data.iloc[50000:,:]
B_test=B_data.iloc[50000:,:]
C_test=C_data.iloc[50000:,:]
y=data['class']

Index(['A', 'B', 'C', 'class'], dtype='object')


In [17]:
A_train.iloc[0,:]

A        02461
class        1
Name: 0, dtype: object

In [3]:
from tensorflow.keras import layers
from tensorflow.keras import Model
def make_model(input_shape):
    x = layers.Conv2D(16, 3, activation='relu')(input_shape)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Conv2D(32, 3, activation='relu')(x)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Conv2D(64, 3, activation='relu')(x)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Flatten()(x)
    model = Model(input_shape, x)
    return model

In [4]:
inputa = keras.Input(shape=(450,300,3))
inputb = keras.Input(shape=(450,300,3))
inputc = keras.Input(shape=(450,300,3))
modela=make_model(inputa)
modelb=make_model(inputb)
modelc=make_model(inputc)
combined = tf.keras.layers.concatenate([modela.output, modelb.output,modelc.output])
end = layers.Dense(512, activation='relu')(combined)
output = layers.Dense(1, activation='sigmoid')(end)
model = Model(inputs=[modela.input, modelb.input,modelc.input], outputs=output)
model.summary()


Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 450, 300, 3) 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 450, 300, 3) 0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 450, 300, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 448, 298, 16) 448         input_1[0][0]                    
____________________________________________________________________________________________

In [5]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
input_imgen = ImageDataGenerator(rescale = 1./255)

test_imgen = ImageDataGenerator(rescale = 1./255)



def generate_generator_multiple(generator,dir1, dir2,dir3, batch_size):
    genX1 = generator.flow_from_dataframe(dir1,directory=r'C:\Users\Ramon\Documents\ETH\Master\2. semester\Introduction to machine learning\task 4\food\food\\', x_col="A", y_col="class",
                                          target_size = (450,300),
                                          class_mode = 'categorical',
                                          batch_size = batch_size,
                                          shuffle=False, 
                                          seed=7,validate_filenames=False,drop_duplicates=False)
    
    genX2 = generator.flow_from_dataframe(dir2,directory=r'C:\Users\Ramon\Documents\ETH\Master\2. semester\Introduction to machine learning\task 4\food\food\\', x_col="B", y_col="class",
                                          target_size = (450,300),
                                          class_mode = 'categorical',
                                          batch_size = batch_size,
                                          shuffle=False, 
                                          seed=7,validate_filenames=False,drop_duplicates=False)
    genX3 = generator.flow_from_dataframe(dir3,directory=r'C:\Users\Ramon\Documents\ETH\Master\2. semester\Introduction to machine learning\task 4\food\food\\', x_col="C", y_col="class",
                                          target_size = (450,300),
                                          class_mode = 'categorical',
                                          batch_size = batch_size,
                                          shuffle=False, 
                                          seed=7,validate_filenames=False,drop_duplicates=False)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            X3i = genX3.next()
            yield [X1i[0], X2i[0],X3i[0]], X2i[1]  #Yield both images and their mutual label
            
            
inputgenerator=generate_generator_multiple(generator=input_imgen,
                                           dir1=A_train,
                                           dir2=B_train,
                                           dir3=C_train,
                                           batch_size=32)       
     
testgenerator=generate_generator_multiple(test_imgen,
                                          dir1=A_test,
                                          dir2=B_test,
                                          dir3=C_test,
                                          batch_size=32)              
          
history=model.fit_generator(inputgenerator,
                        epochs = 10,
                        steps_per_epoch=100,
                        validation_data = testgenerator,
                        shuffle=False,verbose=1)

Found 50000 non-validated image filenames belonging to 2 classes.
Found 50000 non-validated image filenames belonging to 2 classes.
Found 50000 non-validated image filenames belonging to 2 classes.
Epoch 1/10
 27/100 [=======>......................] - ETA: 15:28 - loss: 0.6936 - acc: 0.5000